In [1]:
import sys
sys.path.append('..')
from economic_brazil.economic_data_brazil import data_economic
from economic_brazil.codigos_graficos import Graficos
from economic_brazil.tratando_economic_brazil import tratando_dados_bcb, tratando_dados_ibge_link, tratando_metas_inflacao, tratando_dados_expectativas, tratando_dados_ibge_codigos



In [2]:
tratando_dados_bcb()

,selic
Date,
2000-01-01,18.94
2000-02-01,18.87
2000-03-01,18.85
2000-04-01,18.62
2000-05-01,18.51
...,...
2023-12-01,11.87
2024-01-01,11.65
2024-02-01,11.15


In [4]:
data_economic(salvar=True, formato="csv",diretorio="dados/economic_data_brazil.csv")

OSError: Cannot save file into a non-existent directory: 'dados'

In [3]:
dados = data_economic()

In [ ]:
graficos = Graficos()

In [ ]:
graficos.plotar_temporal(dados=dados)

In [ ]:
graficos.plotar_heatmap(dados)

In [ ]:
graficos.go_plotar(dados)

In [ ]:
dados